# Install the required Dependency

1. Transformers: Hugging face's library for fine-tuning pre-trained BERT models
2. VncoreNLP: Vietnamese NLP Toolkit from the author of PhoBERT
3. Visen: Clean tone for Vietnamese sentences

In [1]:
!pip install transformers==4.12.5 --quiet
!pip install py_vncorenlp --quiet
!pip install sentencepiece --quiet
!pip install tokenizer --quiet
!pip install vncorenlp
!pip install -q emoji
!pip install visen
!pip install pip install multiprocesspandas
!git clone https://github.com/vncorenlp/VnCoreNLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=bf4ae3e8e1bb1837d159bd71c828a8dac4d2390b35e1ec76f03170b557e1b021
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 533.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for bogo: filename=bogo-1.1-py3-none-any.whl size=28150 sha256=f1a9fcf71637f0a5489432df869407db8104fa8b995210fc3f80eb1823c8152b
  Stored in directory: /root/.cache/pip/wheels/fa/ee/4f/64cfdd5fa844d5565d679243dcbfd89e2c8f1ccb6f3d489fb8
Successfully built bogo
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 852.2 kB/s eta 0:00:00
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.13
    Uninst

In [2]:
import json
import re
import string

import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE

from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# import py_vncorenlp
# import VnCoreNLP
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch import optim

from transformers import *
from tqdm.auto import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

import visen
from multiprocesspandas import applyparallel

# Import The Dataset

The original dataset has some third language reviews such as Japanese and Chinese. Therefore, we has hand-translated all those reviews including those in the test dataset.

In [3]:
train_normal = pd.read_csv('/kaggle/input/foody-data/train-translate.csv')
train_normal = train_normal[['Comment', 'Rating']]
train_normal

,Comment,Rating
0,"Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trô...",1
1,Gọi ship 1 xuất cari gà bánh naan và 3 miếng g...,0
2,"Thời tiết lạnh như này, cả nhà rủ nhau đến leg...",1
3,Em có đọc review thấy mng bảo trà sữa nướng đề...,0
4,"Đồ ăn rất ngon, nhà hàng cũng rất đẹp, tất cả ...",1
...,...,...
9136,Mình qua quán này cũng tầm 3-4 lần rồi. Tối qu...,1
9137,quán ở vị trí kim cương\n4-5 tầng gì đó mà diệ...,1
9138,Pizza ngon nhưng cái nước màu xanh kia không k...,1
9139,Nghe chỗ này bán nem chua rán ngon lắm nê...,1


We also crawl 20k more data on the Foody website using Scrapy and Selenium. However, we only select some appropriate records for training and validating

In [4]:
train_crawl = pd.read_csv('/kaggle/input/foody-data/more-train.csv')
train_crawl

,Comment,Rating
0,"Mọi thứ cốt bánh, ruốc, sốt đều ổn.\nNhưng cái...",0.0
1,"Từng là khách quen của quán, trước đó đồ ăn rấ...",0.0
2,Mình có order 1 hộp sữa chua từ chè Xoan. Dù h...,0.0
3,"Gà trong lon, không có hộp, bất tiện cho dân đ...",0.0
4,Hôm nay mình có đặt 1 suất bánh đa trộn đầy đủ...,0.0
...,...,...
953,Chỉ thic ăn pizza ở đây thôi. Các vị bò rất ng...,1.0
954,"Ăn ở đây đc 2 lần, lần nào cx ổn. Ngon, đần đặ...",1.0
955,"Khá ngon,chuẩn bi hang nhanh,size 20 bé quá hic",1.0
956,Bt vẫn hay ăn ở đây bánh khá hợp khẩu vị,1.0


In [5]:
train_set = pd.concat([train_normal, train_crawl], ignore_index=True)
train_set = train_set.drop_duplicates(subset='Comment', keep="first")
train_set = train_set.dropna()
train_set

,Comment,Rating
0,"Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trô...",1.0
1,Gọi ship 1 xuất cari gà bánh naan và 3 miếng g...,0.0
2,"Thời tiết lạnh như này, cả nhà rủ nhau đến leg...",1.0
3,Em có đọc review thấy mng bảo trà sữa nướng đề...,0.0
4,"Đồ ăn rất ngon, nhà hàng cũng rất đẹp, tất cả ...",1.0
...,...,...
10094,Chỉ thic ăn pizza ở đây thôi. Các vị bò rất ng...,1.0
10095,"Ăn ở đây đc 2 lần, lần nào cx ổn. Ngon, đần đặ...",1.0
10096,"Khá ngon,chuẩn bi hang nhanh,size 20 bé quá hic",1.0
10097,Bt vẫn hay ăn ở đây bánh khá hợp khẩu vị,1.0


In [6]:
train_set["Rating"].value_counts()

1.0    7819
0.0    2253
Name: Rating, dtype: int64

In [7]:
with open('/kaggle/input/foody-data/hanoi.json', 'r') as f:
  data_foody_crawl = json.loads(f.read())

val_data = pd.json_normalize(data_foody_crawl)
val_data.loc[val_data['Rating'].astype(float) >= 6, 'Tag'] = 1
val_data.loc[val_data['Rating'].astype(float) < 6, 'Tag'] = 0
val_data['Rating'] = val_data['Tag']
val_data = val_data[['Comment', 'Rating']]
val_data = val_data.drop_duplicates(subset='Comment', keep="first")
val_data = val_data.dropna()
val_data

,Comment,Rating
0,Đây là quán bún mọc Sơn Hà. Vì quán gần học vi...,1.0
1,quán siêu siêu gần nhà thấy lúc nào cũng đông ...,1.0
2,"Từ hồi còn học cấp 1 cấp 2, đây là quán cực cự...",0.0
3,Món ăn ở đây thật tuyệt vời,1.0
4,"Mình gọi 1 suất miến xào lòng nhưng quán hết, ...",0.0
...,...,...
1695,✅ Vị trí: \nQuán này có vẻ mở lâu r nhưng giờ ...,1.0
1696,(+) Điểm cộng :) :\n _ Giá cả phải chăng\n...,1.0
1697,Không gian quán hơi bé nằm trên mặt đường quán...,1.0
1698,✅ Vị trí: \nCũng gần nhà mà không thuận đường ...,1.0


In [8]:
val_data["Rating"].value_counts()

1.0    1303
0.0     387
Name: Rating, dtype: int64

In [9]:
train_data = train_set.drop_duplicates(subset='Comment', keep="first")
train_data = train_data.dropna()
train_data.shape

(10072, 2)

In [10]:
test_data = pd.read_csv('/kaggle/input/foody-data/test_translate.csv')
test_data["Comment"].fillna('ngon', inplace=True)

# Preprocess & Clean Data

In [11]:
from vncorenlp import VnCoreNLP

rdrsegmenter = VnCoreNLP('/kaggle/working/VnCoreNLP/VnCoreNLP-1.1.1.jar', annotators="wseg", max_heap_size='-Xmx500m')

We clean the text data by removing punctuation, html tags, url links, emails and hashtag since these does not contribute much to the context of the sentence

In [12]:
def remove_num(text):
  text = re.sub(r'\w*\d\w*', '', text).strip()
  return text

def remove_punctuation(text):
	text = re.sub(r'[^\w\s]', ' ', text)
	return text

def remove_html_url(text):
  html_pattern = re.compile('<.*?>')
  url_pattern = re.compile(r'https?://\S+|www\.\S+')
  text = html_pattern.sub(r' ', text)
  text = url_pattern.sub(r' ', text)
  text = text.replace("\n", ". ")
  text = text.replace("\r", "")
  text = text.replace("\t", "")
  return text

def remove_email(text):
  required_output = re.sub(r'[A-Za-z0-9]*@[A-Za-z]*\.?[A-Za-z0-9]*', "", text)
  required_output=" ".join(required_output.split())
  return required_output

def remove_hashtag(text):
  clean_tweet = re.sub("#[A-Za-z0-9_]+","", text)
  return clean_tweet

Secondly, We also did some replacement for emojis and acrobyms and also segement words by using the VNCoreNLP library

In [13]:
import emoji

replace_list_1 = pd.read_csv('/kaggle/input/foody-data/acrobyms.csv', header=None, dtype={0: str}).set_index(0).squeeze().to_dict()

replace_list_2 = {
    'ô kêi': 'ok', 'okie': 'ok', 'o kê': 'ok', 'okey': 'ok', 'ôkê': 'ok', 'oki': 'ok', 'oke': 'ok', 'okay': 'ok', 'okê': 'ok', 'ojke': 'ok',
    'okk' : 'ok', 'okey' : 'ok', 'okeyy' : 'ok', 'okela' : 'ok', 'okee' : 'ok', 'okeee' : 'ok', 'okeeee' : 'ok',

    'tks': 'cảm ơn', 'thks': 'cảm ơn', 'thanks': 'cảm ơn', 'ths': 'cảm ơn', 'thank': 'cảm ơn',
    'kg': 'không', 'not': 'không', 'k': 'không', 'kh': 'không', 'kô': 'không', 'hok': 'không', 'ko': 'không', 'khong': 'không', 'kp': 'không phải',
    'kbh': 'không bao giờ', 'kb': 'không biết', 'ntn': 'như thế nào',
    'cute': 'dễ thương', 'khs': 'không hiểu sao', 'nhìu': 'nhiều',
    'hịn': 'xịn', 't_t': 'tiêu cực', 'đếi' : 'đấy', 'nha' : 'nhé', 'nhaaa' : 'nhé', 'hehehehe' : 'cười', 'hehehehe' : 'cười', 'iu' : 'yêu',

    'sz': 'cỡ', 'size': 'cỡ', 
    'wa': 'quá', 'wá': 'quá', 'qá': 'quá', 
    'đx': 'được', 'dk': 'được', 'dc': 'được', 'đk': 'được', 'đc': 'được', 'đuqowjc': 'được',
    'vs': 'với', 'j': 'gì', '“': ' ', 'time': 'thời gian', 'm': 'mình', 'mik': 'mình', 'mk': 'mình', 'r': 'rồi', 'bjo': 'bao giờ', 'very': 'rất',
    'nhưg': 'nhưng', 'cx': 'cũng', 'mng': 'mọi người', 'mn': 'mọi người', 'j': 'gì', 'ah' : 'à', 'cũg' : 'cũng',
    'bh': 'bao giờ', 'km' : 'khuyến mại', 'ms': 'mới', 

    'shop': 'cửa hàng', 'menu': 'thực đơn', 'nc': 'nước', 'nchung': 'nói chung', 'ncl': 'nói chung là', 'ngta': 'người ta', 'sv' : 'sinh viên',
    'sp': 'sản phẩm', 'product': 'sản phẩm', 'hàg': 'hàng', 'nvien' : 'nhân viên', 
    'ship': 'giao hàng', 'delivery': 'giao hàng', 'síp': 'giao hàng', 'order': 'đặt hàng', 'ord': 'đặt hàng', 'shiper': 'người giao hàng',
    'shipper': 'người giao hàng', 'qyán' : 'quán', 'siu' : 'siêu', 'oder' : 'order', 'mỳ' : 'mì', 'cmt' : 'bình luận', 'trog' : 'trong', 'ns' : 'nói', 'cg' : 'cũng',
    'uốnh' : 'uống', 'xuýt' : 'suýt',

    'gud': 'tốt', 'wel done': 'tốt', 'good': 'tốt', 'gút': 'tốt', 'tot': 'tốt', 'nice': 'tốt', 'perfect': 'rất tốt', 
    'quality': 'chất lượng', 'chất lg': 'chất lượng', 'chat': 'chất', 'excelent': 'hoàn hảo',
    'sad': 'tệ', 'por': 'tệ', 'poor': 'tệ', 'bad': 'tệ',
    'beautiful': 'đẹp tuyệt vời', 'dep': 'đẹp', 
    'xau': 'xấu', 'sấu': 'xấu',
    'ngonnnnnnnnnmmmmmmnmmmmmmmmmmmmmnnn' : 'ngon', 'ngonnn' : 'ngon', 'ưnggg' : 'ưng', 'ưngg' : 'ưng',
     
    'thik': 'thích', 'iu': 'yêu', 'fake': 'giả mạo', 'thíc': 'thích', 'thík': 'thích', 'thic': 'thíc',
    'quickly': 'nhanh', 'quick': 'nhanh', 'fast': 'nhanh',
    'fresh': 'tươi', 'delicious': 'ngon', 'p': 'phải', 'mlem': 'ngon', 'bthuong': 'bình thường',

    'dt': 'điện thoại', 'fb': 'facebook', 'face': 'facebook', 'ks': 'khách sạn', 'nv': 'nhân viên',
    'nt': 'nhắn tin', 'ib': 'nhắn tin', 'tl': 'trả lời', 'trl': 'trả lời', 'rep': 'trả lời', 'rv': 'review',
    'fback': 'feedback', 'fedback': 'feedback', 'fbker' : 'facebooker', 'faceboker' : 'facebooker',
    'sd': 'sử dụng', 'sài': 'xài',

    'うどん': 'mì udon',
    'お勘定まで訂正が無い': 'Không sửa tài khoản',
    'そのためお料理まで美味しく感じなくなりました': 'Kết quả là món ăn không còn ngon nữa',
    'その後はボトルの白ワインをオーダーしました': 'Sau đó tôi gọi một chai rượu vang trắng',
    'とコチラに': 'và đây',
    'のような盛り付けも素敵': 'Sắp xếp như',
    'はまま': '',
    'もう少し従業員の教育をすべきです': 'Bạn nên giáo dục nhân viên của mình nhiều hơn một chút',
    'オーダーミスに始まり': 'Tất cả bắt đầu với một sai lầm đặt hàng',
    'ガイドブックを見て気になっていたレトロな雰囲気の蟹鍋屋さん': 'Một quán lẩu riêu cua với không gian hoài cổ thu hút sự chú ý của tôi khi đọc sách hướng dẫn',
    'コスパよかった': 'Hiệu suất chi phí là tốt',
    'コチラの本店をえらびました': 'Tôi đã chọn cửa hàng chính ở đây',
    'スタッフもみんな親切で': 'Tất cả nhân viên đều thân thiện',
    'スープ': 'súp',
    'チキンに添えてあるココナッツ風味の揚げたbanh': 'Bánh gà chiên cốt dừa',
    'テーブルセッティングされた食器がとても可愛いかったです': 'Bộ đồ ăn đặt trên bàn rất dễ thương.',
    'ハノイ最終日の夜なので綺麗で美味しいレストランで': 'Đêm cuối Hà Nội nên quán đẹp và ngon',
    'ハーフサイズでこのボリュームです': 'Một nửa kích thước và khối lượng này',
    'パスターブリューイングのクラフトビールで乾杯して': 'Chúc mừng bia thủ công của Pasta Brewing',
    'パラパラの玉子チャーハンに海老の濃厚なスープをかけて食べるスタイルです': 'Đó là phong cách ăn cơm chiên với trứng bông và súp tôm đặc.',
    'ピリ辛でレモングラスが効いていたのでトムヤムクンを優しくしたような味でした': 'Nó có vị cay và sả rất hiệu quả, vì vậy nó có vị như tom yum goong với hương vị nhẹ nhàng.',
    'ㅋㅋㅋ': 'cười',
    '上显示的食物图片': 'hình ảnh thực phẩm hiển thị trên',
    '不要为了利益': 'không vì lợi nhuận',
    '也请商家和foody': 'Hãy cũng thương nhân và foody',
    '今回の旅行で鍋を食べてなかったので': 'Tôi đã không ăn nabe trong chuyến đi này',
    '但是却不太愿意为我们送上食物': 'nhưng không sẵn sàng phục vụ thức ăn cho chúng tôi',
    '但请不要忽悠任何消费者的权利': 'Nhưng xin đừng đánh lừa bất kỳ quyền lợi nào của người tiêu dùng',
    '価格もローカルフードにしては高めの価格でした': 'Giá cũng cao cho thực phẩm địa phương.',
    '写真に写ってませんが鍋にはたっぷりのブンも付きます': 'Nó không được hiển thị trong ảnh, nhưng nồi đi kèm với rất nhiều bánh.',
    '前回も頼んだチキンのグリルと茄子のグリルも安定の美味しさ': 'Món gà nướng và cà tím nướng lần trước tôi gọi cũng rất ngon.',
    '前回品切れしていた豚の角煮が食べれて嬉しい': 'Tôi rất vui vì có thể ăn món thịt kho đã hết hàng lần trước.',
    '勿体ない': 'lãng phí',
    '叉燒': 'nướng',
    '友達と二人で265000': '26k5 với một người bạn',
    '合わせて行かれるのをオススメします': 'Tôi khuyên bạn nên đi cùng nhau',
    '和商家的广告存在极高的欺骗成分': 'Có một mức độ lừa dối rất cao trong các quảng cáo của các thương gia và thương gia',
    '在那里却没有供应': 'có nhưng không có nguồn cung cấp',
    '地酒も美味しい': 'Rượu sake địa phương cũng ngon',
    '場所もいいので': 'vị trí tốt',
    '定価は35000vndでも': 'Dù giá niêm yết là 35k',
    '心地よい空間でした': 'Đó là một không gian thoải mái.',
    '我们不求吃得好': 'chúng tôi không muốn ăn ngon',
    '揚げネギがたっぷりのったスティッキーライスとセットになっていてご飯と一緒に食べるとまた美味しい': 'Xôi được ăn kèm với nhiều hành lá phi thơm, ăn với cơm rất ngon.',
    '昨晚和朋友在这里用餐': 'Ăn tối ở đây với bạn bè đêm qua',
    '服务人员态度极差': 'Nhân viên phục vụ rất tệ',
    '楽lên': 'dễ',
    '次回はアメリカ牛肉のすき焼きを食べてみよう': 'Lần sau thử món sukiyaki bò Mỹ nhé',
    '欺骗消费者': 'lừa người tiêu dùng',
    '注意': 'Để ý',
    '海老の味が濃く': 'Vị tôm đậm đà',
    '海鮮鍋もオーダーしました': 'Tôi cũng gọi món hải sản.',
    '深夜営業が許されているtong': 'tong được phép hoạt động vào đêm khuya',
    '炒めた海老のタレをかけて完成する味付けです': 'Nêm mắm tôm xào là hoàn thành',
    '牛肉の肉うどん': 'mì bò',
    '玉子チャーハンは味が薄く': 'Cơm chiên trứng có vị nhạt',
    '絶品海老チャーハンのお店と有名なローカルレストラン': 'Cửa hàng cơm chiên tôm tuyệt vời và nhà hàng địa phương nổi tiếng',
    '给我们一大堆借口': 'cho chúng tôi một loạt các lý do',
    '美味しい': 'ngon',
    '美味しいじゃなくてもとても高いです': 'Không ngon nhưng rất đắt',
    '肉の量がとにかく多かった': 'có quá nhiều thịt',
    '茄子の味付けが以前より少し甘味が強く感じました': 'Gia vị của cà tím cảm thấy ngọt hơn trước một chút.',
    '虽说是自助餐': 'Dù là tiệc buffet',
    '蟹の素揚げはインスタ映え': 'Cua chiên giòn có thể Instagram được',
    '贅沢な味わい': 'hương vị sang trọng',
    '近くにハノイ鉄道の線路があるので': 'Vì gần đó có tuyến đường sắt Hà Nội',
    '酒飲みのアテにピーナッツ': 'Đậu phộng cho dân nhậu',
    '鍋はスタッフさんが一人前すつ取り分けてくれます': 'Nhân viên sẽ lấy từng chiếc nabe ra.',
    '食べたかったティンリーの炒め物も食べれて嬉しい': 'Tôi rất vui vì đã được ăn món chiên của Tinley mà tôi muốn ăn.',
    '香港牛腩面秘': 'Bí mật mì ức bò Hồng Kông',
    '騙す': 'lừa dối',
    '鳥肉の唐揚げはレモングラスの風味がとても美味しい': 'Thịt gà chiên giòn có hương vị sả thơm ngon.',
    'ꈍᴗꈍ': 'ngượng ngùng',
    '눈_눈': 'không hài lòng',
    '일요일': 'Chủ nhật',
    '콩카페': 'cà phê đậu',
}
emojis = {
    '😂' : ' tích cực ', '👍' : ' tích cực ', '👍🏻' : ' tích cực ', '👍🏼' : ' tích cực ', '👍🏽' : ' tích cực ', 
    '❤' : ' tích cực ', '😍' : ' tích cực ', '🤣' : ' tích cực ', '👌' : ' tích cực ', '😋' : ' tích cực ', 
    '😅' : ' tích cực ', '😁' : ' tích cực ', '🥰' : ' tích cực ', '🤤' : ' tích cực ', '😆' : ' tích cực ', 
    '☺' : ' tích cực ', '😊' : ' tích cực ', '😌' : ' tích cực ', '😉' : ' tích cực ', '👏' : ' tích cực ', 
    '💯' : ' tích cực ', '🤗' : ' tích cực ', '😝' : ' tích cực ', '😄' : ' tích cực ', '😀' : ' tích cực ',
    '✅' : ' tích cực ', '💋' : ' tích cực ', '😚' : ' tích cực ', '✌' : ' tích cực ', '💓' : ' tích cực ', 
    '🤩' : ' tích cực ', '💕' : ' tích cực ', '♥' : ' tích cực ', '😛' : ' tích cực ', '😜' : ' tích cực ', 
    '💪' : ' tích cực ', '🤪' : ' tích cực ', '😎' : ' tích cực ', '😗' : ' tích cực ', '🙆' : ' tích cực ', 
    '🔥' : ' tích cực ', '💛' : ' tích cực ', '😻' : ' tích cực ', '😙' : ' tích cực ', '😹' : ' tích cực ', 
    '🥳' : ' tích cực ', '❣' : ' tích cực ', '♡' : ' tích cực ', '🖤' : ' tích cực ', '💗' : ' tích cực ', 
    '🤭' : ' tích cực ', '💘' : ' tích cực ', '💜' : ' tích cực ', '💙' : ' tích cực ', '💞' : ' tích cực ', 
    '💝' : ' tích cực ', '💟' : ' tích cực ', '🧡' : ' tích cực ', '😘' : ' tích cực ', '👌🏻' : ' tích cực ', 
    '😡' : ' tiêu cực ', '🤦' : ' tiêu cực ', '🥲' : ' tiêu cực ', '🤬' : ' tiêu cực ', '😨' : ' tiêu cực ', 
    '👎' : ' tiêu cực ', '💔' : ' tiêu cực ', '🤮' : ' tiêu cực ', '🤢' : ' tiêu cực ', 
    '🙂' : ' ', '😃' : ' ', 
    '😭' : ' ', '😢' : ' ', '😞' : ' ', '🙏' : ' ', '🥺' : ' ', '😥' : ' ',
    '🤔' : ' ', '💰' : ' ', '💸' : ' ', '😑' : ' ', '😪' : ' ',
    '😤' : ' ', '😕' : ' ', '😱' : ' ', '😳' : ' ', '😩' : ' ', '😨' : ' ',
    '😔' : ' ', '🙁' : ' ', '☹' : ' ', '😓' : ' ', '🙃' : ' ', '😠' : ' ', '😣' : ' ',
    '📍' : ' ', '👉' : ' ', '✔' : ' ', '⭐' : ' ', '✨' : ' ', '🏠' : ' ', '🐽' : ' ', '📌' : ' ',
    '🍴' : ' ', '🍹' : ' ', '🌟' : ' ', '⏰' : ' ',
    '🙄' : ' ', '😒' : ' ', '🌸' : ' ', '😬' : ' ', 
}

replace_list_1.update(replace_list_2)

def replace_emoji(text):
    pattern = re.compile("|".join(emojis.keys()))
    text = pattern.sub(lambda m: emojis[re.escape(m.group(0))], text)
    return text

def normalize_acronyms(text):
    words = []
    for word in text.strip().split():
        word = word.strip(string.punctuation)
        if word.lower() not in replace_list_1.keys(): words.append(word)
        else: words.append(replace_list_1[word.lower()])
    return emoji.demojize(' '.join(words)) # Remove Emojis


def word_segmentation(text):
    return ' '.join([' '.join(sen) for sen in rdrsegmenter.tokenize(text)])



def remove_space(text):
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
    return text

In [14]:
def text_preprocess(text):
    text = remove_html_url(text)
    text = remove_email(text)
    text = remove_hashtag(text)
    text = remove_punctuation(text)
#     text = remove_num(text)
    text = visen.clean_tone(text)
    text = replace_emoji(text)
    text = normalize_acronyms(text)
    text = word_segmentation(text)
    text = remove_space(text)
    return text.lower()

In [15]:
text_preprocess("Xôi dẻo, đồ ăn đậm vị")

'xôi dẻo đồ_ăn đậm vị'

In [16]:
train_data['Comment'] = train_data['Comment'].apply(text_preprocess)
val_data['Comment'] = val_data['Comment'].apply(text_preprocess)
test_data['Comment'] = test_data['Comment'].apply(text_preprocess)

In [17]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.99M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Train & Evaluate Models

### Set Up

1. Seeding for comparision between models
2. Hyperparameters settings

In [18]:
torch.manual_seed(10)
torch.cuda.manual_seed(10)

It is recommended by the author of BERT to fine-tune BERT with the following parameters:
1. Batch size: [16, 32]
2. Learning rate: [1e-5, 2e-5, 3e-5, 5e-5]

In [19]:
# hyperparameters
MAX_LEN = 256
BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 2e-5

Our model was train on GPU P100 to get the results on the Kaggle competition

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
def encoding(data, tokenizer, max_token_len=256):
  comments = data.Comment
  input_ids = []
  attention_masks = []

  for index, comment in enumerate(comments):
    encoded = tokenizer.encode_plus(
      comment,
      add_special_tokens=True,
      max_length=max_token_len,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    input_ids.append(encoded["input_ids"])
    attention_masks.append(encoded["attention_mask"])

  input_ids = torch.cat(input_ids,dim=0)
  attention_masks = torch.cat(attention_masks,dim=0)
  return input_ids, attention_masks

### Models Overview
1. PhoBert-based

PhoBERT is the first large-scale monolingual pre-trained language model for Vietnamese, was introduced in 2020. The architecture of our fine-tuned PhoBERT is shown below

![bertbase.png](https://firebasestorage.googleapis.com/v0/b/e-learning-2497f.appspot.com/o/bertbase.png?alt=media&token=20a8b2a5-6f5a-44e1-be8b-f6b124448880)

In [22]:
class BertBase(nn.Module):

  def __init__(self, n_classes, drop_out=0.1):
    super(BertBase, self).__init__()
    self.bert = AutoModel.from_pretrained("vinai/phobert-base")
    self.classifier = nn.Sequential(
        nn.Dropout(drop_out),
        nn.Linear(self.bert.config.hidden_size, n_classes),
        nn.Sigmoid()
    )

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.bert(input_ids, attention_mask=attention_mask)
    return self.classifier(output.pooler_output)

2. PhoBERT+LSTM
We also fine-tune PhoBERT with other heads for all tokens classification such as LSTM. The PhoBERT+LSTM architecture is represented as following

![bertlstm.png](https://firebasestorage.googleapis.com/v0/b/e-learning-2497f.appspot.com/o/bertlstm.png?alt=media&token=00b4255f-3972-4ffa-bf18-ae5013099708)

In [23]:
class BertLSTM(nn.Module):

  def __init__(self, n_classes, drop_out=0.1):
    super(BertLSTM, self).__init__()
    self.bert = AutoModel.from_pretrained("vinai/phobert-base")
    self.lstm = nn.LSTM(768, 128, bidirectional=True, batch_first=True)
    self.dropout = nn.Dropout(drop_out)
    self.fc = nn.Linear(256, n_classes)
    self.sigmoid = nn.Sigmoid()

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.bert(input_ids, attention_mask=attention_mask)
    output = output[0]
    enc_hiddens, (last_hidden, last_cell) = self.lstm(output)
    output = torch.cat((last_hidden[0], last_hidden[1]), dim=1)
    output = self.dropout(output)
    output = self.fc(output)    
    return self.sigmoid(output)

3. PhoBERT + TextCNN 

We combined PhoBERT with Text Convolutional neural networks (TextCNN) which was introduced by Yoon Kim in 2014. The PhoBERT+CNN architecture is represented as following

![bertcnn.png](https://firebasestorage.googleapis.com/v0/b/e-learning-2497f.appspot.com/o/bertcnn.png?alt=media&token=0f4f313c-9b17-4321-976e-be501924ef8c)

In [24]:
class BertCNN(nn.Module):
  def __init__(self, embedding_dim=768, n_filters=64, filter_sizes=[1, 2, 3, 5], output_dim=1, drop_out=0.1):
    super(BertCNN, self).__init__()
    self.bert = AutoModel.from_pretrained("vinai/phobert-base")
    self.convs1 = nn.ModuleList([nn.Conv2d(1, n_filters, (K, embedding_dim)) for K in filter_sizes])
    self.dropout = nn.Dropout(drop_out)
    self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
    self.sigmoid = nn.Sigmoid()

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.bert(input_ids, attention_mask=attention_mask)
    output = output[0].unsqueeze(0).permute(1, 0, 2, 3)
    output = [F.relu(conv(output)) for conv in self.convs1]
    output = [F.max_pool1d(i.squeeze(-1), i.size(2)).squeeze(2) for i in output]  
    output = torch.cat(output, 1)
    output = self.dropout(output)
    output = self.fc(output)    
    return self.sigmoid(output)

### Training & Validating Phrase

In [25]:
def train_epoch(model, optimizer, train_loader):
    model.train()
    total_loss = total = 0
    total_correct = 0
    progress_bar = tqdm(train_loader, desc='Training', leave=False)
    predictions = []
    labels = []
    for batch in progress_bar:

        label = batch[2].to(device)
        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)

        # Clean old gradients
        optimizer.zero_grad()

        # Forwards pass
        output = model(input_ids, attention_masks).squeeze(dim=1)

        # Calculate how wrong the model is
        loss = criterion(output, label)
        
        numpy_logits = output.cpu().detach().numpy()
        predictions += list(np.where(numpy_logits >= 0.5, 1.0, 0.0))
        labels += list(label.cpu().detach().numpy())

        # Perform gradient descent, backwards pass
        loss.backward()

        # Take a step in the right direction
        optimizer.step()
        

        # Record metrics
        total_loss += loss.item()
        total += len(label)

        progress_bar.set_postfix({'Training_loss': '{:.3f}'.format(loss.item()/len(batch))}, refresh=False)

    return accuracy_score(predictions, labels), total_loss / total


def validate_epoch(model, valid_loader):
    model.eval()
    total_loss = total = 0
    total_correct = 0
    predictions = []
    labels = []
    prob = []
    with torch.no_grad():
        progress_bar = tqdm(valid_loader, desc='Validating', leave=False)
        for batch in progress_bar:
            label = batch[2].to(device)
            input_ids = batch[0].to(device)
            attention_masks = batch[1].to(device)

            # Forwards pass
            output = model(input_ids, attention_masks).squeeze(dim=1)

            # Calculate how wrong the model is
            loss = criterion(output, label)

            numpy_logits = output.cpu().detach().numpy()
            prob += list(numpy_logits)
            predictions += list(np.where(numpy_logits >= 0.5, 1.0, 0.0))
            labels += list(label.cpu().detach().numpy())

            # Record metrics
            total_loss += loss.item()
            total += len(label)

    roc_auc = roc_auc_score(labels, prob)
    f1 = f1_score(labels, predictions)
    precision = precision_score(labels, predictions)
    recall = recall_score(labels, predictions)
    print(classification_report(labels, predictions, digits=4))
    return accuracy_score(predictions, labels), total_loss / total, roc_auc, f1, precision, recall


def predict_test(model, test_loader):
    model.eval()
    predictions = []
    prob = []
    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc='Validating', leave=False)
        for batch in progress_bar:
            input_ids = batch[0].to(device)
            attention_masks = batch[1].to(device)

            # Forwards pass
            output = model(input_ids, attention_masks)

            numpy_logits = output.cpu().detach().numpy()
            predictions += list(np.where(numpy_logits >= 0.5, 1.0, 0.0))
            prob += list(numpy_logits)

    return predictions, prob

In [26]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import TensorDataset, random_split

# splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=42).split(df["Comment"], df["Rating"]))

# for fold, (train_idx, val_idx) in enumerate(splits):
#     print("Training for fold {}".format(fold))

#     if fold != 0:
#       continue

train_input_ids, train_attention_masks = encoding(train_data, tokenizer, max_token_len=MAX_LEN)
val_input_ids, val_attention_masks = encoding(val_data, tokenizer, max_token_len=MAX_LEN)
train_encoded_label_tensors = torch.FloatTensor(np.array(train_data.Rating))
val_encoded_label_tensors = torch.FloatTensor(np.array(val_data.Rating))
    
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_encoded_label_tensors)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_encoded_label_tensors)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

model = BertBase(n_classes=1)
model = model.to(device)
criterion = nn.BCELoss()

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
]

optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE, eps=1e-8)
    
num_training_steps = EPOCHS * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)
    
best_f1 = 0

for epoch in range(EPOCHS):

    train_acc, train_loss = train_epoch(model, optimizer, train_dataloader)
    valid_acc, valid_loss, roc_auc, f1, precision, recall = validate_epoch(model, val_dataloader)
    
    if f1 > best_f1:
        best_f1 = f1
        print(best_f1)
        torch.save({'epoch': epoch,
                      'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                      'loss': criterion,
                      }, f'/kaggle/working/bert_base.pth')

    print('Epoch {}/{}'.format(epoch, EPOCHS - 1))
    print('-' * 10)
    print('Training Loss: {:.2e} Acc: {:.8f}'.format(train_loss, train_acc))
    print('Validate Loss: {:.2e} Acc: {:.8f}'.format(valid_loss, valid_acc))
    print('Precision Score: {: .8f}'.format(precision))
    print('Recall Score: {: .8f}'.format(recall))
    print('ROC AUC Score: {: .8f}'.format(roc_auc))
    print('F1 Score: {: .8f}'.format(f1))
    print()

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Training:   0%|          | 0/315 [00:00<?, ?it/s]

Validating:   0%|          | 0/53 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         0.0     0.8916    0.7649    0.8234       387
         1.0     0.9330    0.9724    0.9523      1303

    accuracy                         0.9249      1690
   macro avg     0.9123    0.8686    0.8878      1690
weighted avg     0.9235    0.9249    0.9228      1690

0.9522735813603909
Epoch 0/4
----------
Training Loss: 8.45e-03 Acc: 0.89872915
Validate Loss: 6.47e-03 Acc: 0.92485207
Precision Score:  0.93298969
Recall Score:  0.97237145
ROC AUC Score:  0.95718487
F1 Score:  0.95227358



...

Validating:   0%|          | 0/53 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         0.0     0.8267    0.8630    0.8445       387
         1.0     0.9588    0.9463    0.9525      1303

    accuracy                         0.9272      1690
   macro avg     0.8928    0.9047    0.8985      1690
weighted avg     0.9285    0.9272    0.9278      1690

Epoch 3/4
----------
Training Loss: 4.66e-03 Acc: 0.95254170
Validate Loss: 6.56e-03 Acc: 0.92721893
Precision Score:  0.95878694
Recall Score:  0.94627782
ROC AUC Score:  0.96218427
F1 Score:  0.95249131



Training:   0%|          | 0/315 [00:00<?, ?it/s]

Validating:   0%|          | 0/53 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         0.0     0.8448    0.8579    0.8513       387
         1.0     0.9576    0.9532    0.9554      1303

    accuracy                         0.9314      1690
   macro avg     0.9012    0.9055    0.9033      1690
weighted avg     0.9318    0.9314    0.9315      1690

Epoch 4/4
----------
Training Loss: 3.93e-03 Acc: 0.96167593
Validate Loss: 6.76e-03 Acc: 0.93136095
Precision Score:  0.95759445
Recall Score:  0.95318496
ROC AUC Score:  0.95806636
F1 Score:  0.95538462



# Submission

In [27]:
test_input_ids, test_attention_masks = encoding(test_data, tokenizer, max_token_len=MAX_LEN)

In [28]:
from torch.utils.data import TensorDataset, random_split

test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [29]:
model = BertBase(n_classes=1)
best_model_cp = torch.load('/kaggle/working/bert_base.pth')
model.load_state_dict(best_model_cp['model_state_dict'])
model = model.to(device)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
predictions, prob = predict_test(model, test_dataloader)

Validating:   0%|          | 0/160 [00:00<?, ?it/s]

In [31]:
guess = pd.DataFrame()
guess['RevId'] = test_data["RevId"]
guess['Rating'] = list(map(float, prob))
guess

,RevId,Rating
0,781115,0.468980
1,1219481,0.076715
2,1703765,0.973797
3,4870346,0.019138
4,2638711,0.967530
...,...,...
5098,1025826,0.974589
5099,1278470,0.973449
5100,2565212,0.009540
5101,3766155,0.956680


In [32]:
guess.to_csv('/kaggle/working/prediction.csv', index=False)